## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

Web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled List of Falcon 9 and Falcon Heavy launches

https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922

### Import libraries

In [12]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Functions to process web scraped HTML table

In [30]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

### Request the falcon9 wiki page

In [14]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [16]:
# use requests.get() method with the provided static_url
data = requests.get(static_url).text

In [18]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data,"html5lib")

In [20]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.

In [23]:
first_launch_table = html_tables[2]

Iterate through the <th> elements and apply the provided extract_column_from_header() to extract column name one by one

In [34]:
column_names = []

elements = first_launch_table.find_all('th')
for element in elements:
    name = extract_column_from_header(element)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [36]:
#Check the extracted column names
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Create an empty dictionary with keys from the extracted column names. Later, this dictionary will be converted into a Pandas dataframe

In [42]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Fill up the launch_dict with launch records extracted from table rows

In [49]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            time = datatimelist[1]
            launch_dict['Time'].append(time)

            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
           
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
           
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            customer = row[6]
            if not(customer):
                customer = row[6].a.string
            launch_dict['Customer'].append(customer)
            
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
           
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [51]:
#After filling in the parsed launch record values into launch_dict, create a dataframe from it
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [53]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,NASA,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45
2,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,No attempt\n,8 December 2010,15:43
3,3,CCAFS,Dragon,525 kg,LEO,NASA,Success\n,F9 v1.0B0005.1,No attempt,22 May 2012,07:44
4,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt\n,8 October 2012,00:35


In [55]:
#Save as csv
df.to_csv('spacex_web_scraped.csv', index=False)